In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)
%matplotlib inline

from sklearn.model_selection import train_test_split#, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
import xgboost as xgb

import pickle

from time import time

In [5]:
df = pd.read_csv("features_clean.csv").iloc[:,1:]

In [8]:
df.head()

,flag,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,avg val received,min val sent,max val sent,avg val sent,total tnx,total ether sent,total ether received,total ether balance,total erc20 tnxs,erc20 total ether received,erc20 total ether sent,erc20 total ether sent contract,erc20 uniq sent addr,erc20 uniq rec addr,erc20 uniq sent contract addr,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name,erc20 most sent token type,erc20 most rec token type,sent rec balance
0,0,844.26,1093.71,704785.63,721,89,0,40,118,0.000000,45.806785,6.589513,0.00,31.220000,1.200681,810,865.691093,586.466675,-279.224419,265.0,3.558854e+07,3.560317e+07,0.0,30.0,54.0,0.0,58.0,0.0,1.500000e+07,265586.147600,0.000000,1.683100e+07,271779.920000,39.0,57.0,1,1,8.101124
1,0,12709.07,2958.44,1218216.73,94,8,0,5,14,0.000000,2.613269,0.385685,0.00,1.800000,0.032844,102,3.087297,3.085478,-0.001819,8.0,4.034283e+02,2.260809e+00,0.0,1.0,5.0,0.0,7.0,0.0,3.650000e+02,57.632615,2.260809,2.260809e+00,2.260809,1.0,7.0,1,1,11.750000
2,0,246194.54,2434.02,516729.30,2,10,0,10,2,0.113119,1.165453,0.358906,0.05,3.538616,1.794308,12,3.588616,3.589057,0.000441,8.0,5.215121e+02,0.000000e+00,0.0,0.0,7.0,0.0,8.0,0.0,4.428198e+02,65.189009,0.000000,0.000000e+00,0.000000,0.0,8.0,0,1,0.200000
3,0,10219.60,15785.09,397555.90,25,9,0,7,13,0.000000,500.000000,99.488840,0.00,450.000000,70.001834,34,1750.045862,895.399559,-854.646303,14.0,1.711105e+04,1.141223e+04,0.0,2.0,11.0,0.0,11.0,0.0,1.141223e+04,1555.550174,100.000000,9.029231e+03,3804.076893,1.0,11.0,1,1,2.777778
4,0,36.61,10707.77,382472.42,4598,20,1,7,19,0.000000,12.802411,2.671095,0.00,9.000000,0.022688,4619,104.318883,53.421897,-50.896986,42.0,1.628297e+05,1.235399e+05,0.0,4.0,23.0,0.0,27.0,0.0,9.000000e+04,4934.232147,0.000000,4.500000e+04,13726.659220,6.0,27.0,1,1,229.900000


In [26]:
x, xho, y, yho = train_test_split(df.iloc[:,1:], df.flag, test_size=0.2, random_state=2018)
xtr, xte, ytr, yte = train_test_split(x, y, test_size=0.25, random_state=2019)

In [23]:
dflr = df.fillna(df.median())

In [24]:
xl, xlho, yl, ylho = train_test_split(dflr.iloc[:,1:], dflr.flag, test_size=0.2, random_state=2018)
xltr, xlte, yltr, ylte = train_test_split(xl, yl, test_size=0.25, random_state=2019)

In [71]:
scorers = {
            'fbeta_score':make_scorer(fbeta_score),
            'precision_score': make_scorer(precision_score),
            'recall_score': make_scorer(recall_score),
            'accuracy_score': make_scorer(accuracy_score)
          }

# skf = StratifiedKFold(n_splits=5, shuffle = True)
kf = KFold(n_splits=5, shuffle=True, random_state = 2020)

### Logistic Regression

In [75]:
logreg = LogisticRegression()

scaler = StandardScaler()
xtr_scaled = scaler.fit_transform(xltr.values)
xte_scaled = scaler.transform(xlte.values)
xho_scaled = scaler.transform(xlho.values)

cval = np.logspace(0.001,10,1)

param_grid = dict(C=cval)

gridlog = GridSearchCV(logreg, param_grid, cv=10, scoring=scorers, refit='precision_score', n_jobs=-1)
gridlog.fit(xl, yl)

# to continue with xte_scaled and xho_scaled

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.00230524])}, refit='recall_score',
             scoring={'accuracy_score': make_scorer(accuracy_score),
                      'fbeta_score': make_scorer(fbeta_score),
                      'precision_score': make_scorer(precision_score),
                      'recall_score': make_scorer(recall_score)})

In [76]:
print("Best params: ", gridlog.best_params_)
print("Best estimator: ", gridlog.best_estimator_)
print("Best score: ", gridlog.best_score_)

Best params:  {'C': 1.0023052380778996}
Best estimator:  LogisticRegression(C=1.0023052380778996)
Best score:  nan


In [98]:
gridlog.best_index_

0

### KNN

In [34]:
knn = KNeighborsClassifier()

In [105]:
t0 = time()

# k_range = np.arange(3,100,10)
k_range = np.arange(3,24,5)
weight_options = ['uniform', 'distance']
param_grid = dict(n_neighbors=k_range, weights=weight_options)

gridknn = GridSearchCV(knn, param_grid, cv=10, scoring='precision')
gridknn.fit(xl, yl)

knntime = time() - t0
print(time() - t0)

6.902899980545044


In [41]:
print("Best params: ", gridknn.best_params_)
print("Best estimator: ", gridknn.best_estimator_)
print("Best score: ", gridknn.best_score_)

Best params:  {'n_neighbors': 13, 'weights': 'distance'}
Best estimator:  KNeighborsClassifier(n_neighbors=13, weights='distance')
Best score:  0.8946754509651382


In [38]:
knn_preds = gridknn.predict(xlte)

### Random Forest

In [45]:
rf = RandomForestClassifier(bootstrap=True)
# rfxbs = RandomForestClassifier(bootstrap=False)

In [106]:
t0 = time()

# nest = np.arange(1,302,50)
# nest = np.arange(150,250,25)
nest = np.arange(160,170,2)

param_grid = dict(n_estimators=nest)

gridrf = GridSearchCV(rf, param_grid, cv=10, scoring='precision')
gridrf.fit(xl, yl)

rftime = time() - t0
print(time() - t0)

55.99232888221741


In [53]:
print("Best params: ", gridrf.best_params_)
print("Best estimator: ", gridrf.best_estimator_)
print("Best score: ", gridrf.best_score_)

Best params:  {'n_estimators': 165}
Best estimator:  RandomForestClassifier(n_estimators=165)
Best score:  0.9671128659476118


### Extra Trees

In [56]:
et = ExtraTreesClassifier()

In [107]:
t0 = time()

# nest_et = np.arange(1,302,50) #51
# nest_et = np.arange(1,102,25) $51
nest_et = np.arange(1,26,2)

param_grid = dict(n_estimators=nest_et)

gridet = GridSearchCV(et, param_grid, cv=10, scoring='precision')
gridet.fit(xl, yl)

ettime = time() - t0
print(time() - t0)

5.729576826095581


In [65]:
print("Best params: ", gridet.best_params_)
print("Best estimator: ", gridet.best_estimator_)
print("Best score: ", gridet.best_score_)

Best params:  {'n_estimators': 15}
Best estimator:  ExtraTreesClassifier(n_estimators=15)
Best score:  0.9654051104262968


### XGBoost

In [94]:
gbm = xgb.XGBClassifier(early_stopping_rounds = 10, eval_metric = "logloss", n_estimator = 50)

# eval_set=[(xtr, ytr),(xte, yte)]

params = {
    "max_depth" : np.arange(3,8,2),
    "learning_rate" : np.logspace(.001,1),
    "subsample" : np.arange(.4,1,.1),
    "min_child_weight" : np.linspace(3,13,3),
    "colsample_bytree" : np.arange(.7,1,.1),
    "scale_pos_weight" : np.arange(1,2.1,.3),
}

In [108]:
t0 = time()

gridxgb = RandomizedSearchCV(gbm, param_distributions=params, cv=10, scoring='precision')
gridxgb.fit(x, y)

xgbtime = time() - t0
print(time() - t0)

[22:34:36] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:34:36] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier 

[22:34:36] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:34:36] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/metrics

[22:34:36] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:34:36] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




...

[22:34:54] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:34:54] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

[22:34:54] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:34:54] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:34:55] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimator" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


19.25837206840515


In [109]:
print(knntime)
print(rftime)
print(ettime)
print(xgbtime)

6.902875900268555
55.99230694770813
5.72955584526062
19.25834894180298


### Future work - Voting

In [99]:
pickle.dump(gridknn.best_estimator_, open("knn.pickle", "wb"))
pickle.dump(gridrf.best_estimator_, open("rf.pickle", "wb"))
pickle.dump(gridet.best_estimator_, open("et.pickle", "wb"))
pickle.dump(gridxgb.best_estimator_, open("xgb.pickle", "wb"))

,flag,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,avg val received,min val sent,max val sent,avg val sent,total tnx,total ether sent,total ether received,total ether balance,total erc20 tnxs,erc20 total ether received,erc20 total ether sent,erc20 total ether sent contract,erc20 uniq sent addr,erc20 uniq rec addr,erc20 uniq sent contract addr,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name,erc20 most sent token type,erc20 most rec token type,sent rec balance
7637,1,1457.31,34.12,4815.43,3,13,0,10,3,1.000000,2.501052,1.348445,1.000875,11.277872,5.842916,16,17.528747,17.529785,0.001037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.230769
7638,1,3976.50,834.77,9622.53,2,2,0,1,2,0.586269,0.947515,0.766892,0.585408,0.947284,0.766346,4,1.532693,1.533785,0.001092,1.0,1.337000,0.0000,0.0,0.0,1.0,0.0,1.0,1.337,1.337,1.337000,0.0,0.0000,0.000000,0.0,1.0,1,1,1.000000
7639,1,112.90,31.87,321.42,2,3,0,3,1,0.001020,0.817800,0.439607,0.500390,0.817506,0.658948,5,1.317896,1.318820,0.000924,1.0,1.337000,0.0000,0.0,0.0,1.0,0.0,1.0,1.337,1.337,1.337000,0.0,0.0000,0.000000,0.0,1.0,1,1,0.666667
7640,1,2300.37,65.10,73091.00,29,98,0,89,26,0.000784,15.729068,0.383225,0.000000,36.700000,1.314957,127,38.133767,37.556046,-0.577721,96.0,142677.382900,120354.7684,0.0,6.0,55.0,0.0,37.0,0.000,26436.081,1954.484697,0.0,81324.0746,5232.816018,22.0,37.0,1,1,0.295918
7641,1,0.00,0.00,0.00,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9811,1,12635.10,631.39,58748.48,4,13,0,11,4,0.004082,12.000000,2.821060,0.465461,12.200000,9.166365,17,36.665461,36.673777,0.008316,2.0,1.337000,0.0000,0.0,0.0,2.0,0.0,2.0,0.000,1.337,0.668500,0.0,0.0000,0.000000,0.0,2.0,1,1,0.307692
9812,1,0.00,0.00,0.00,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.0,13.370000,0.0000,0.0,0.0,1.0,0.0,1.0,13.370,13.370,13.370000,0.0,0.0000,0.000000,0.0,1.0,1,1,0.000000
9813,1,2499.44,2189.29,261601.88,67,43,0,31,44,0.001078,21.310000,1.234192,0.004000,9.500000,0.922179,110,61.785995,53.070252,-8.715743,5.0,1503.469504,0.0000,0.0,0.0,5.0,0.0,5.0,0.000,1500.000,300.693901,0.0,0.0000,0.000000,0.0,5.0,1,1,1.558140
9814,1,0.00,0.00,0.00,0,1,0,1,0,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,1,0.000000,0.500000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.000000
